# Enunciado

A partir do arquivo Brasileirao.xlsx que contém os resultados dos jogos do Brasileirão 2016 gerar:

* Tabela de classificação dos mandantes

* Tabela de classificação dos visitantes

* Tabela de classificação geral.

E exportar como um único excel ao final.

# Imports

In [1]:
import pandas as pd

Carregar o arquivo excel como um dataframe

In [2]:
jogos = pd.read_excel("dados/Brasileirao.xlsx")

Tratar o resultado com vários checks booleanos, transformando em inteiro, criando uma coluna para cada check booleano.

In [3]:
jogos["vitoria_casa"] = jogos["Gols Mandante"] > jogos["Gols Visitante"]
jogos["vitoria_fora"] = jogos["Gols Mandante"] < jogos["Gols Visitante"]
jogos["Empates"] = jogos["Gols Mandante"] == jogos["Gols Visitante"]
jogos["wo"] = jogos["Gols Mandante"] == "WO"

Converter colunas em colunas de inteiros para poder agregar.

In [4]:
jogos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 9 columns):
Data              380 non-null datetime64[ns]
Mandante          380 non-null object
Gols Mandante     380 non-null object
Gols Visitante    380 non-null object
Visitante         380 non-null object
vitoria_casa      380 non-null bool
vitoria_fora      380 non-null bool
Empates           380 non-null bool
wo                380 non-null bool
dtypes: bool(4), datetime64[ns](1), object(4)
memory usage: 16.4+ KB


In [5]:
jogos["Gols Mandante"] = pd.to_numeric(jogos["Gols Mandante"], errors='coerce')
jogos["Gols Visitante"] = pd.to_numeric(jogos["Gols Visitante"], errors='coerce')

In [6]:
jogos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 9 columns):
Data              380 non-null datetime64[ns]
Mandante          380 non-null object
Gols Mandante     379 non-null float64
Gols Visitante    379 non-null float64
Visitante         380 non-null object
vitoria_casa      380 non-null bool
vitoria_fora      380 non-null bool
Empates           380 non-null bool
wo                380 non-null bool
dtypes: bool(4), datetime64[ns](1), float64(2), object(2)
memory usage: 16.4+ KB


Agrupar a classificação mandante e visitante.

In [7]:
classificação_mandante = jogos.groupby("Mandante").agg(sum)
classificação_visitante = jogos.groupby("Visitante").agg(sum)


Renomear as colunas para fazerem sentido no contexto

In [8]:
classificação_mandante.rename(index=str, columns=
                              {"Gols Mandante": "Gols Pro", 
                               "Gols Visitante": "Gols Contra",
                               "vitoria_casa": "Vitórias",
                               "vitoria_fora": "Derrotas",
                                "Mandante": "Clube"},inplace=True)
classificação_mandante.index.name = "Clube"

classificação_visitante.rename(index=str, columns=
                              {"Gols Mandante": "Gols Contra", 
                               "Gols Visitante": "Gols Pro",
                               "vitoria_casa": "Derrotas",
                               "vitoria_fora": "Vitórias",
                              },inplace=True)
classificação_visitante.index.name = "Clube"


Gerar colunas de pontos, jogos e saldo de gols utilizando somas e multiplicações.

In [9]:
classificação_mandante["Pontos"] = 3*classificação_mandante["Vitórias"]+classificação_mandante["Empates"] 
classificação_mandante["Jogos"] = classificação_mandante["Vitórias"]+classificação_mandante["Empates"]+classificação_mandante["Derrotas"]+classificação_mandante["wo"] 
classificação_mandante["Saldo"] = classificação_mandante["Gols Pro"] - classificação_mandante["Gols Contra"] 

classificação_visitante["Pontos"] = 3*classificação_visitante["Vitórias"]+classificação_visitante["Empates"] 
classificação_visitante["Jogos"] = classificação_visitante["Vitórias"]+classificação_visitante["Empates"]+classificação_visitante["Derrotas"]+classificação_visitante["wo"] 
classificação_visitante["Saldo"] = classificação_visitante["Gols Pro"] - classificação_visitante["Gols Contra"] 

Reordenar as colunas para ficar mais bonito

In [10]:
nova_ordem_colunas = ['Jogos','Vitórias','Empates','Derrotas','wo','Gols Pro','Gols Contra',"Saldo",'Pontos']
classificação_mandante = classificação_mandante[nova_ordem_colunas]
classificação_visitante = classificação_visitante[nova_ordem_colunas]


Gerar a classificação final.

In [11]:
classificação = classificação_mandante+classificação_visitante

Ordenar as classificações

In [12]:
ordem_desempate = ["Pontos","Vitórias","Saldo","Gols Pro"]
classificação.sort_values(ordem_desempate,ascending=False,inplace=True)
classificação_mandante.sort_values(ordem_desempate,ascending=False,inplace=True)
classificação_visitante.sort_values(ordem_desempate,ascending=False,inplace=True)


Mostrar as classificações

# Classificação Mandante

In [13]:
classificação_mandante

,Jogos,Vitórias,Empates,Derrotas,wo,Gols Pro,Gols Contra,Saldo,Pontos
Clube,,,,,,,,,
Atletico-PR,19.0,15.0,3.0,1.0,0.0,27.0,6.0,21.0,48.0
Santos,19.0,15.0,2.0,2.0,0.0,36.0,11.0,25.0,47.0
Palmeiras,19.0,14.0,4.0,1.0,0.0,35.0,13.0,22.0,46.0
Atletico-MG,19.0,13.0,3.0,3.0,0.0,39.0,20.0,19.0,42.0
Ponte Preta,19.0,13.0,2.0,4.0,0.0,33.0,18.0,15.0,41.0
Flamengo RJ,19.0,12.0,5.0,2.0,0.0,31.0,16.0,15.0,41.0
Gremio,19.0,11.0,4.0,4.0,0.0,22.0,11.0,11.0,37.0
Corinthians,19.0,10.0,7.0,2.0,0.0,27.0,11.0,16.0,37.0
Sport Recife,19.0,10.0,5.0,4.0,0.0,33.0,22.0,11.0,35.0


# Classificação Visitante

In [14]:
classificação_visitante

,Jogos,Vitórias,Empates,Derrotas,wo,Gols Pro,Gols Contra,Saldo,Pontos
Clube,,,,,,,,,
Palmeiras,19.0,10.0,4.0,5.0,0.0,27.0,19.0,8.0,34.0
Flamengo RJ,19.0,8.0,6.0,5.0,0.0,21.0,19.0,2.0,30.0
Santos,19.0,7.0,3.0,9.0,0.0,23.0,24.0,-1.0,24.0
Botafogo RJ,19.0,7.0,3.0,9.0,0.0,16.0,22.0,-6.0,24.0
Cruzeiro,19.0,7.0,2.0,10.0,0.0,19.0,23.0,-4.0,23.0
Chapecoense-SC,19.0,5.0,7.0,7.0,0.0,24.0,32.0,-8.0,22.0
Atletico-MG,20.0,4.0,9.0,6.0,1.0,22.0,30.0,-8.0,21.0
Fluminense,19.0,5.0,5.0,9.0,0.0,15.0,21.0,-6.0,20.0
Sao Paulo,19.0,5.0,5.0,9.0,0.0,15.0,23.0,-8.0,20.0


# Classificação Geral

In [15]:
classificação

,Jogos,Vitórias,Empates,Derrotas,wo,Gols Pro,Gols Contra,Saldo,Pontos
Clube,,,,,,,,,
Palmeiras,38.0,24.0,8.0,6.0,0.0,62.0,32.0,30.0,80.0
Santos,38.0,22.0,5.0,11.0,0.0,59.0,35.0,24.0,71.0
Flamengo RJ,38.0,20.0,11.0,7.0,0.0,52.0,35.0,17.0,71.0
Atletico-MG,39.0,17.0,12.0,9.0,1.0,61.0,50.0,11.0,63.0
Botafogo RJ,38.0,17.0,8.0,13.0,0.0,43.0,39.0,4.0,59.0
Atletico-PR,38.0,17.0,6.0,15.0,0.0,38.0,32.0,6.0,57.0
Corinthians,38.0,15.0,10.0,13.0,0.0,48.0,42.0,6.0,55.0
Ponte Preta,38.0,15.0,8.0,15.0,0.0,48.0,52.0,-4.0,53.0
Gremio,38.0,14.0,11.0,13.0,0.0,41.0,44.0,-3.0,53.0


Exportar resultado em um único excel

In [16]:
writer = pd.ExcelWriter('Tabelas Gabarito/BR2016_Simples.xlsx')
classificação_mandante.to_excel(writer,'Classificação Mandante')
classificação_visitante.to_excel(writer,'Classificação Visitante')
classificação.to_excel(writer,'Classificação Geral')
writer.close()